# An Analysis

This notebook will be rendered on the website. Below is an example plot.

In [24]:
from openfisca_us import Microsimulation, IndividualSim

In [25]:
from openfisca_us.model_api import *
from openfisca_us.variables.household.demographic.tax_unit.filing_status import (
    FilingStatus,
)
from openfisca_us.tools.baseline_variables import baseline_variables

REPLACED_PROGRAMS = ["ctc", "eitc"]

CHILD_AMOUNT = 393
CHILD_AGE_LIMIT = 18
ADULT_DEPENDENT_CREDIT = 600
SINGLE_FILER_CREDIT = 600
MARRIED_FILER_CREDIT = 1200
PHASE_OUT_START_SINGLE = 20_000
PHASE_OUT_START_MARRIED = 40_000
PHASE_OUT_RATE = 0.05

class ecpa(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act"

    def formula(tax_unit, period, parameters):
        person = tax_unit.members
        qualifies_for_child_allowance = person("age", period) <= CHILD_AGE_LIMIT
        child_allowance = CHILD_AMOUNT * tax_unit.sum(qualifies_for_child_allowance)

        # TODO: check if 18 year old tax filer or spouse of household is eligible
        adult_dependent = person("is_tax_unit_dependent", period) & ~qualifies_for_child_allowance
        dependent_credit = ADULT_DEPENDENT_CREDIT * tax_unit.sum(adult_dependent)

        filing_status = tax_unit("filing_status", period)
        is_married = filing_status == filing_status.possible_values.JOINT
        max_filer_credit = where(is_married, MARRIED_FILER_CREDIT, SINGLE_FILER_CREDIT)

        agi = tax_unit("adjusted_gross_income", period)
        phase_out_start = where(is_married, PHASE_OUT_START_MARRIED, PHASE_OUT_START_SINGLE)

        excess = max_(agi - phase_out_start, 0)
        reduction = excess * PHASE_OUT_RATE
        filer_credit = max_(max_filer_credit - reduction, 0)

        return (child_allowance * 12) + dependent_credit + filer_credit

class income_tax(Variable):
    def formula(tax_unit, period, parameters):
        original = baseline_variables["income_tax"].formula(tax_unit, period)
        return original - tax_unit("ecpa", period)

class add_ecpa(Reform):
    def apply(self):
        for program in REPLACED_PROGRAMS:
            self.neutralize_variable(program)
        self.add_variable(ecpa)
        self.update_variable(income_tax)

In [26]:
# ms = Microsimulation(year=2020)
isb = IndividualSim(year=2022)
isr = IndividualSim(add_ecpa, year=2022)

for individual_sim in [isb, isr]:
    individual_sim.add_person(name="parent1", age=25)
    # individual_sim.add_person(name="parent2", age=25)
    individual_sim.add_person(name="child1", age=5)
    individual_sim.add_person(name="child2", age=5)
    members = ["parent1", "parent2", "child1", "child2"]
    members = ["parent1", "child1", "child2"]
    individual_sim.add_tax_unit(
        name="tax_unit",
        members=members,
        premium_tax_credit=0,
    )
    individual_sim.add_spm_unit(
        name="spm_unit",
        members=members,
    )
    individual_sim.add_household(
        name="household",
        members=members,
        state_code="CA",
    )
    individual_sim.vary("employment_income")

In [43]:
import pandas as pd

df = pd.DataFrame(
    dict(
        employment_income=isb.calc("employment_income")[0],
        net_income_baseline=isb.calc("spm_unit_net_income")[0],
        net_income_ecpa=isr.calc("spm_unit_net_income")[0],
        mtr_baseline=1 - isb.deriv("spm_unit_net_income", "employment_income", wrt_target="parent1"),
        mtr_ecpa=1 - isr.deriv("spm_unit_net_income", "employment_income", wrt_target="parent1"),
    )
)

df["diff"] = df["net_income_ecpa"] - df["net_income_baseline"]
df["mtr_diff"] = df["mtr_ecpa"] - df["mtr_baseline"]

In [41]:
import plotly.express as px

px.line(df, x="employment_income", y=["net_income_baseline", "net_income_ecpa"])

In [34]:
px.line(df, x="employment_income", y="diff")

In [42]:
fig = px.line(
    df,
    x="employment_income",
    y=["mtr_baseline", "mtr_ecpa"]
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".1%",
    yaxis_range=[-1, 1],
)
fig.show()

In [44]:
px.line(df, x="employment_income", y="mtr_diff")